https://zhuanlan.zhihu.com/p/75054200
## 1. 中间梯度的查询

在 PyTorch 的计算图（computation graph）中，只有叶子结点（leaf nodes）的变量会保留梯度。
而所有中间变量的梯度只被用于反向传播，一旦完成反向传播，中间变量的梯度就将自动释放，从而节约内存。





## 1.1 两种方式实现查询：
1) 在反向传播前使用retain_grad -- 该方法耗内存
2) 使用hook机制 -- 推荐:  hook_fn(grad) -> Tensor or None

注意: 虽然是不同的实现方式,但是对于节点来说两者都是注册在`Tensor._backward_hooks`字典里
参考: [完全理解Pytorch里面的Hook机制教学视频](https://www.bilibili.com/video/BV1MV411t7td?from=search&seid=4733570378288382581)


In [1]:



import torch

x = torch.Tensor([0, 1, 2, 3]).requires_grad_()
y = torch.Tensor([4, 5, 6, 7]).requires_grad_()
w = torch.Tensor([1, 2, 3, 4]).requires_grad_()
z = x+y
# z.retain_grad()

o = w.matmul(z)
o.backward()  
# o.retain_grad()

print('x.requires_grad:', x.requires_grad) # True
print('y.requires_grad:', y.requires_grad) # True
print('z.requires_grad:', z.requires_grad) # True
print('w.requires_grad:', w.requires_grad) # True
print('o.requires_grad:', o.requires_grad) # True


print('x.grad:', x.grad) # tensor([1., 2., 3., 4.])
print('y.grad:', y.grad) # tensor([1., 2., 3., 4.])
print('w.grad:', w.grad) # tensor([ 4.,  6.,  8., 10.])
print('z.grad:', z.grad) # None
print('o.grad:', o.grad) # None

x.requires_grad: True
y.requires_grad: True
z.requires_grad: True
w.requires_grad: True
o.requires_grad: True
x.grad: tensor([1., 2., 3., 4.])
y.grad: tensor([1., 2., 3., 4.])
w.grad: tensor([ 4.,  6.,  8., 10.])
z.grad: None
o.grad: None


In [3]:
# retain_grad

import torch

x = torch.Tensor([0, 1, 2, 3]).requires_grad_()
y = torch.Tensor([4, 5, 6, 7]).requires_grad_()
w = torch.Tensor([1, 2, 3, 4]).requires_grad_()
z = x+y
z.retain_grad()

o = w.matmul(z)
o.retain_grad()  # 在反向传播前进行retain_grad就可以得到保留梯度，但是这种方式很耗内存
o.backward()


print('x.requires_grad:', x.requires_grad) # True
print('y.requires_grad:', y.requires_grad) # True
print('z.requires_grad:', z.requires_grad) # True
print('w.requires_grad:', w.requires_grad) # True
print('o.requires_grad:', o.requires_grad) # True


print('x.grad:', x.grad) # tensor([1., 2., 3., 4.])
print('y.grad:', y.grad) # tensor([1., 2., 3., 4.])
print('w.grad:', w.grad) # tensor([ 4.,  6.,  8., 10.])
print('z.grad:', z.grad) # None
print('o.grad:', o.grad) # None

x.requires_grad: True
y.requires_grad: True
z.requires_grad: True
w.requires_grad: True
o.requires_grad: True
x.grad: tensor([1., 2., 3., 4.])
y.grad: tensor([1., 2., 3., 4.])
w.grad: tensor([ 4.,  6.,  8., 10.])
z.grad: tensor([1., 2., 3., 4.])
o.grad: tensor(1.)


In [12]:

# 使用hook
import torch

x = torch.Tensor([0, 1, 2, 3]).requires_grad_()
y = torch.Tensor([4, 5, 6, 7]).requires_grad_()
w = torch.Tensor([1, 2, 3, 4]).requires_grad_()
z = x+y

# ===================
def hook_fn(grad):
    print("grad: ", grad)

z.retain_grad()
zhook = z.register_hook(hook_fn)
z.retain_grad()

print(z._backward_hooks)  
## 注意: 尽管上面的z有三个hook,但是每一个hook加入的时候都是按照字典的规则进行加入的,即
##  不会重复添加同一种hook
# ===================

o = w.matmul(z)
ohook = o.register_hook(hook_fn)

print('=====Start backprop=====')
o.backward()  # 此时hook自动执行
print('=====End backprop=====')

print('x.grad:', x.grad)
print('y.grad:', y.grad)
print('w.grad:', w.grad)
print('z.grad:', z.grad)

zhook.remove()
ohook.remove()  # 在使用完hook后别忘记了释放掉,除非后续还要用

OrderedDict([(22, <function Tensor.retain_grad.<locals>.retain_grad_hook at 0x0000024BB06A6F28>), (23, <function hook_fn at 0x0000024BB06F27B8>)])
=====Start backprop=====
grad:  tensor(1.)
grad:  tensor([1., 2., 3., 4.])
=====End backprop=====
x.grad: tensor([1., 2., 3., 4.])
y.grad: tensor([1., 2., 3., 4.])
w.grad: tensor([ 4.,  6.,  8., 10.])
z.grad: tensor([1., 2., 3., 4.])



## 2. hook改变中间梯度值

In [5]:

import torch
import torch

x = torch.Tensor([0, 1, 2, 3]).requires_grad_()
y = torch.Tensor([4, 5, 6, 7]).requires_grad_()
w = torch.Tensor([1, 2, 3, 4]).requires_grad_()
z = x + y


# ===================
def hook_fn(grad):
    g = 2 * grad
    print(g)
    return g


z.register_hook(hook_fn)
# ===================

o = w.matmul(z)

print('=====Start backprop=====')
o.backward()
print('=====End backprop=====')

print('x.grad:', x.grad)
print('y.grad:', y.grad)
print('w.grad:', w.grad)
print('z.grad:', z.grad)  # 因为z的梯度变为两倍，因此反传的时候前面的梯度也是2倍


=====Start backprop=====
tensor([2., 4., 6., 8.])
=====End backprop=====
x.grad: tensor([2., 4., 6., 8.])
y.grad: tensor([2., 4., 6., 8.])
w.grad: tensor([ 4.,  6.,  8., 10.])
z.grad: None


### 2.0.1 不要再hook里对grad做inplace操作


In [14]:

import torch
a = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

c = a*b 
d = torch.tensor(4., requires_grad=True)

def d_hook(grad):
    
    grad *=100  # 这个grad会影响到c的梯度，从而影响整个grad传播
    
    # return grad * 100  # 这样就不会影响a,b的梯度
d.register_hook(d_hook)

e = c + d
e.backward()

print(a.grad)  # 300
print(b.grad)  # 200




tensor(3.)
tensor(2.)


### 2.1 一个变量同时绑定多个hook

In [8]:
import torch

x = torch.Tensor([0, 1, 2, 3]).requires_grad_()
y = torch.Tensor([4, 5, 6, 7]).requires_grad_()
w = torch.Tensor([1, 2, 3, 4]).requires_grad_()
z = x + y

# ===================
z.register_hook(lambda x: 100.+ x)
z.register_hook(lambda x: print("grad: ", x))
# ===================

o = w.matmul(z)

print('=====Start backprop=====')
o.backward()
print('=====End backprop=====')


=====Start backprop=====
grad:  tensor([101., 102., 103., 104.])
=====End backprop=====



## 3. hook for Module

网络模块 module 不像上一节中的 Tensor，拥有显式的变量名可以直接访问，而是被封装在神经网络中间。我们通常只能获得网络整体的输入和输出，对于夹在网络中间的模块，我们不但很难得知它输入/输出的梯度，甚至连它输入输出的数值都无法获得。除非设计网络时，在 forward 函数的返回值中包含中间 module 的输出，或者用很麻烦的办法，把网络按照 module 的名称拆分再组合，让中间层提取的 feature 暴露出来。

为了解决这个麻烦，PyTorch 设计了两种 hook：register_forward_hook 和 register_backward_hook，分别用来获取正/反向传播时，中间层模块输入和输出的 feature/gradient，大大降低了获取模型内部信息流的难度。


In [ ]:
### 3.2. register pre forward hook

在forward之前的hook


In [22]:
import torch
import torch.nn  as nn

class sumnet(nn.Module):
    def __init__(self):
        super(sumnet, self).__init__()
    
    @staticmethod
    def forward(a,b,c):
        d = a+b+c
        print("a:", a)
        print("b:",b)
        print("c:",c)
        return d

def forward_pre_hook(module, input):
    '''
    The input contains only the positional arguments given to the module. 
    Keyword arguments won’t be passed to the hooks and only to the forward. 
    The hook can modify the input. User can either return a tuple or a single modified value in the hook. We will wrap the value
     into a tuple if a single value is returned(unless that value is already a tuple).
    
    :param module: 
    :param input: 
    :return: 
    '''
    a,b = input  
    
    print("pre b:", b)
    
    return a+10, b # 经过pre_hook后，输入forward的是a = 12, b = 3, c = 4.



def forward_hook(module, inputs, outputs):
    '''
    
    :param module: 
    :param inputs: 接受来自pre_forward_hook的输出！！！ 即只有a, b
    :param outputs: 此时的outputs接受的输入是forward的输出结果
    :return: 返回的值会覆盖forward的输出，送到下一个module里执行
    '''
    
    print("inputs: ",inputs)  # 12
    print("outputs: ",outputs) # 19
    return outputs+100  

sum_net = sumnet()

sum_net.register_forward_pre_hook(forward_pre_hook)
sum_net.register_forward_hook(forward_hook)

a = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)
c = torch.tensor(4., requires_grad=True)

d = sum_net(a,b,c= c)
d.backward()
print(d)

        



(tensor(2., requires_grad=True), tensor(3., requires_grad=True), tensor(4., requires_grad=True))
pre b: tensor(3., requires_grad=True)


TypeError: forward() missing 1 required positional argument: 'c'

### 3.1. register forward hook
在forward之后的hook

register_forward_hook的作用是获取前向传播过程中，各个网络模块的输入和输出。
对于模块 module，其使用方式为：module.register_forward_hook(hook_fn)
hook_fn(module, input, output) -> None

它的输入变量分别为：模块，模块的输入，模块的输出，和对 Tensor 的 hook 不同，forward hook 不返回任何值，
也就是说不能用它来修改输入或者输出的值（注意：从 pytorch 1.2.0 开始，forward hook 也有返回值了，可以修改网络模块的输出），
但借助这个 hook，我们可以方便地用预训练的神经网络提取特征，而不用改变预训练网络的结构。

In [9]:
import torch
from torch import nn

# 首先我们定义一个模型
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(3, 4)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(4, 1)
        self.initialize()
    
    # 为了方便验证，我们将指定特殊的weight和bias
    def initialize(self):
        with torch.no_grad():
            self.fc1.weight = torch.nn.Parameter(
                torch.Tensor([[1., 2., 3.],
                              [-4., -5., -6.],
                              [7., 8., 9.],
                              [-10., -11., -12.]]))

            self.fc1.bias = torch.nn.Parameter(torch.Tensor([1.0, 2.0, 3.0, 4.0]))
            self.fc2.weight = torch.nn.Parameter(torch.Tensor([[1.0, 2.0, 3.0, 4.0]]))
            self.fc2.bias = torch.nn.Parameter(torch.Tensor([1.0]))

    def forward(self, x):
        o = self.fc1(x)
        o = self.relu1(o)
        o = self.fc2(o)
        return o

# 全局变量，用于存储中间层的 feature
total_feat_out = []
total_feat_in = []

# 定义 forward hook function
def hook_fn_forward(module, input, output):
    print(module) # 用于区分模块
    print('input', input) # 首先打印出来
    print('output', output)
    total_feat_out.append(output) # 然后分别存入全局 list 中
    total_feat_in.append(input)


model = Model()

modules = model.named_children() # 
for name, module in modules:
    module.register_forward_hook(hook_fn_forward)

# 注意下面代码中 x 的维度，对于linear module，输入一定是大于等于二维的
# （第一维是 batch size）。

x = torch.Tensor([[1.0, 1.0, 1.0]]).requires_grad_() 
o = model(x)
o.backward()

print('==========Saved inputs and outputs==========')
for idx in range(len(total_feat_in)):
    print('input: ', total_feat_in[idx])
    print('output: ', total_feat_out[idx])

Linear(in_features=3, out_features=4, bias=True)
input (tensor([[1., 1., 1.]], requires_grad=True),)
output tensor([[  7., -13.,  27., -29.]], grad_fn=<AddmmBackward>)
ReLU()
input (tensor([[  7., -13.,  27., -29.]], grad_fn=<AddmmBackward>),)
output tensor([[ 7.,  0., 27.,  0.]], grad_fn=<ReluBackward0>)
Linear(in_features=4, out_features=1, bias=True)
input (tensor([[ 7.,  0., 27.,  0.]], grad_fn=<ReluBackward0>),)
output tensor([[89.]], grad_fn=<AddmmBackward>)
==========Saved inputs and outputs==========
input:  (tensor([[1., 1., 1.]], requires_grad=True),)
output:  tensor([[  7., -13.,  27., -29.]], grad_fn=<AddmmBackward>)
input:  (tensor([[  7., -13.,  27., -29.]], grad_fn=<AddmmBackward>),)
output:  tensor([[ 7.,  0., 27.,  0.]], grad_fn=<ReluBackward0>)
input:  (tensor([[ 7.,  0., 27.,  0.]], grad_fn=<ReluBackward0>),)
output:  tensor([[89.]], grad_fn=<AddmmBackward>)



### 3.2. register backward hook (目前还有bug，不推荐使用)

* 1) 和 register_forward_hook相似，register_backward_hook 的作用是获取神经网络反向传播过程中，
各个模块输入端和输出端的梯度值。对于模块 module，其使用方式为：module.register_backward_hook(hook_fn) 

* 2) 其中hook_fn的函数签名为：

`hook_fn(module, grad_input, grad_output) -> Tensor or None`

它的输入变量分别为：模块，模块输入端的梯度，模块输出端的梯度。
需要注意的是，这里的输入端和输出端，是站在**前向传播**的角度的，而不是反向传播的角度。
例如线性模块：o=W*x+b，其输入端为 W，x 和 b，输出端为 o.

* 3)如果模块有多个输入或者输出的话，grad_input和grad_output可以是 tuple 类型。
对于线性模块：o=W*x+b ，**它的输入端包括了W、x 和 b 三部分，
因此 grad_input 就是一个包含三个元素的 tuple.**
（注意，这里的输入和输出是相对于该模块的，类似的比如在torch.autograd.function.Function
重新定义模块的self.backward的输入就是grad_output，这个是output是上一层对该层的梯度输入，即视角都是前向传播的视角，
grad_input就是该模块梯度输出到上一层模块进行上模块的反传）

和forward hook不同点：

    1. 在forward hook中，hook_fn的输入是x; 不包括w, b, 但是backward hook包括，输入和输出是元组
    2. 返回tensor或者None, backward hook函数不能直接改变它的输入变量, 但是可以返回新的grad_input,反向传播到它上一个模块
        ） 

In [10]:
import torch
from torch import nn


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(3, 4)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(4, 1)
        self.initialize()

    def initialize(self):
        with torch.no_grad():
            self.fc1.weight = torch.nn.Parameter(
                torch.Tensor([[1., 2., 3.],
                              [-4., -5., -6.],
                              [7., 8., 9.],
                              [-10., -11., -12.]]))

            self.fc1.bias = torch.nn.Parameter(torch.Tensor([1.0, 2.0, 3.0, 4.0]))
            self.fc2.weight = torch.nn.Parameter(torch.Tensor([[1.0, 2.0, 3.0, 4.0]]))
            self.fc2.bias = torch.nn.Parameter(torch.Tensor([1.0]))

    def forward(self, x):
        o = self.fc1(x)
        o = self.relu1(o)
        o = self.fc2(o)
        return o


total_grad_out = []
total_grad_in = []


def hook_fn_backward(module, grad_input, grad_output):
    print(module) # 为了区分模块
    # 为了符合反向传播的顺序，我们先打印 grad_output
    print('grad_output', grad_output) 
    # 再打印 grad_input
    print('grad_input', grad_input)
    # 保存到全局变量
    total_grad_in.append(grad_input)
    total_grad_out.append(grad_output)


model = Model()

modules = model.named_children()
for name, module in modules:
    module.register_backward_hook(hook_fn_backward)

# 这里的 requires_grad 很重要，如果不加，backward hook
# 执行到第一层，对 x 的导数将为 None，某英文博客作者这里疏忽了
# 此外再强调一遍 x 的维度，一定不能写成 torch.Tensor([1.0, 1.0, 1.0]).requires_grad_()
# 否则 backward hook 会出问题。
x = torch.Tensor([[1.0, 1.0, 1.0]]).requires_grad_()
o = model(x)
o.backward()

print('==========Saved inputs and outputs==========')
for idx in range(len(total_grad_in)):
    print('grad output: ', total_grad_out[idx])
    print('grad input: ', total_grad_in[idx])

Linear(in_features=4, out_features=1, bias=True)
grad_output (tensor([[1.]]),)
grad_input (tensor([1.]), tensor([[1., 2., 3., 4.]]), tensor([[ 7.],
        [ 0.],
        [27.],
        [ 0.]]))
ReLU()
grad_output (tensor([[1., 2., 3., 4.]]),)
grad_input (tensor([[1., 0., 3., 0.]]),)
Linear(in_features=3, out_features=4, bias=True)
grad_output (tensor([[1., 0., 3., 0.]]),)
grad_input (tensor([1., 0., 3., 0.]), tensor([[22., 26., 30.]]), tensor([[1., 0., 3., 0.],
        [1., 0., 3., 0.],
        [1., 0., 3., 0.]]))
==========Saved inputs and outputs==========
grad output:  (tensor([[1.]]),)
grad input:  (tensor([1.]), tensor([[1., 2., 3., 4.]]), tensor([[ 7.],
        [ 0.],
        [27.],
        [ 0.]]))
grad output:  (tensor([[1., 2., 3., 4.]]),)
grad input:  (tensor([[1., 0., 3., 0.]]),)
grad output:  (tensor([[1., 0., 3., 0.]]),)
grad input:  (tensor([1., 0., 3., 0.]), tensor([[22., 26., 30.]]), tensor([[1., 0., 3., 0.],
        [1., 0., 3., 0.],
        [1., 0., 3., 0.]]))



### 注意事项

register_backward_hook只能操作简单模块，而不能操作包含多个子模块的复杂模块。 
如果对复杂模块用了 backward hook，那么我们只能得到该模块最后一次简单操作的梯度信息。
对于上面的代码稍作修改，不再遍历各个子模块，
而是把 model 整体绑在一个 hook_fn_backward上

